# **4 function和自动图**

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn 
import pandas as pd
import os, gc, sys, time
import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

## **4.1 将python函数转换为图函数**

- tf.function : 将python代码变为tf计算图

In [4]:
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z>0 ? sacle * z : scale * alpha * scale * tf.nn.elu(z)
    is_pos = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_pos, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant([-2., 3])))

tf.Tensor([-0.86466473  3.        ], shape=(2,), dtype=float32)


> 常规python无法不在循环内处理向量

**使用tf.function转换**

In [7]:
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant([-2., 3])))
print(scaled_elu_tf.python_function)

tf.Tensor([-0.86466473  3.        ], shape=(2,), dtype=float32)
<function scaled_elu at 0x000002B626C278C8>


> 通过将python函数转换位tensorflow库函数，能够显著提升运行速度

**使用@tf.function转换**

In [10]:
@ tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment= tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


In [11]:
# 到底是怎么转换的呢
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown(f'```python\n{code}\n```'))

In [12]:
display_tf_code(scaled_elu)

```python
def tf__scaled_elu(z, scale=None, alpha=None):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('scaled_elu', 'scaled_elu_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as scaled_elu_scope:
    is_pos = ag__.converted_call(tf.greater_equal, scaled_elu_scope.callopts, (z, 0.0), None, scaled_elu_scope)
    do_return = True
    retval_ = scaled_elu_scope.mark_return_value(scale * ag__.converted_call(tf.where, scaled_elu_scope.callopts, (is_pos, z, alpha * ag__.converted_call(tf.nn.elu, scaled_elu_scope.callopts, (z,), None, scaled_elu_scope)), None, scaled_elu_scope))
  do_return,
  return ag__.retval(retval_)

```

In [14]:
var = tf.Variable(0.)
@ tf.function
def add_21():
    return var.assign_add(21)
print(add_21())

tf.Tensor(21.0, shape=(), dtype=float32)


> Variable不可以在函数内部创建

## **4.2 函数签名**

In [17]:
@tf.function
def cube(z):
    return tf.pow(z, 3)

print(cube(tf.constant([1, 2, 3])))
print(cube(tf.constant([1., 2., 3.])))

tf.Tensor([ 1  8 27], shape=(3,), dtype=int32)
tf.Tensor([ 1.  8. 27.], shape=(3,), dtype=float32)


*python是一门弱类型的语言，如果函数需要接受什么类型的输入，在函数内部很难去显式指定，导致错误常见, 我们可以使用函数签名指定*

`input_signature`可以指定输入的类型和属性

In [18]:
@tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
def cube(z):
    return tf.pow(z, 3)
try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name='x'))


> @tf.function 将python函数转换位tf计算图      
> get_concrete_function 为函数添加input_signature, 同时使之变成可以保存的model

In [19]:
cube_func_int32 = cube.get_concrete_function(tf.TensorSpec([None], tf.int32, name='x'))
print(cube_func_int32)

In [20]:
# 获取图
cube_func_int32.graph

In [21]:
# 获取图内部的操作
cube_func_int32.graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Pow/y' type=Const>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [22]:
cube_func_int32.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [23]:
cube_func_int32.graph.get_tensor_by_name('x:0')

<tf.Tensor 'x:0' shape=(None,) dtype=int32>

In [24]:
# 查看所有的图结构
cube_func_int32.graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
node {
  name: "Pow/y"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Pow"
  op: "Pow"
  input: "x"
  input: "Pow/y"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Identity"
  op: "Identity"
  input: "Pow"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
versions {
  producer: 119
}

**以上的函数有什么用呢?**
- 模型的保存和载入
- 内部求导计算